In [1]:
import datajoint as dj
import numpy as np
import time
from tqdm import tqdm

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

pinky = dj.create_virtual_module('pinky', 'microns_pinky')
schema = dj.schema('microns_pinky')


Connecting celiib@10.28.0.34:3306


In [3]:
@schema
class LeftoverCoarseLabelFinal(dj.Computed):
    definition = """
    # Ensure that Vertex labels for pinky.LeftoverProofreadLabel match the triangle labels, so these are regenerated from the correct triangle labels.
    -> pinky.LeftoverProofreadLabel
    ---
    vertices  : longblob # Corrected vertex labels
    triangles : longblob # Same triangle labels as pinky.LeftoverProofreadLabel
    """
    
    key_source = pinky.LeftoverProofreadLabel & 'status="complete"'
    
    def make(self, key):
        start = time.time()
        
        print(key["segment_id"])
        labels = (pinky.LeftoverProofreadLabel & key).fetch1()
        corrected_vertex_labels = np.zeros(labels['vertices'].shape, np.uint8)
        
        mesh = (pinky.ExcitatoryLeftoverMeshes & 'decimation_ratio=0.35' & dict(segment_id=key['segment_id'])).fetch1()
        mesh_triangles = mesh['triangles']
        
        vertex_label_dict = dict()
        for i, label in enumerate(labels['triangles']):
            triangle = mesh_triangles[i]
            for node in triangle:
                if node in vertex_label_dict:
                    if vertex_label_dict[node] < label:
                        vertex_label_dict[node] = label
                else:
                    vertex_label_dict[node] = label
                
        for node, label in vertex_label_dict.items():
            corrected_vertex_labels[node] = label
            
        self.insert1(dict(key,
                          vertices=corrected_vertex_labels,
                          triangles=labels['triangles']))
        
        print("Segment {} vertex labels regenerated in: {} seconds.".format(key['segment_id'], time.time() - start))

In [4]:
start = time.time()
LeftoverCoarseLabelFinal.populate(reserve_jobs=True)
print(time.time() - start)

648518346349506617
Segment 648518346349506617 vertex labels regenerated in: 0.33928680419921875 seconds.
648518346349506803
Segment 648518346349506803 vertex labels regenerated in: 0.5390849113464355 seconds.
648518346349507351
Segment 648518346349507351 vertex labels regenerated in: 0.2995133399963379 seconds.
648518346349507788
Segment 648518346349507788 vertex labels regenerated in: 0.3125724792480469 seconds.
648518346349508212
Segment 648518346349508212 vertex labels regenerated in: 0.4144454002380371 seconds.
648518346349508960
Segment 648518346349508960 vertex labels regenerated in: 0.4780416488647461 seconds.
648518346349509393
Segment 648518346349509393 vertex labels regenerated in: 0.3073606491088867 seconds.
648518346349509512
Segment 648518346349509512 vertex labels regenerated in: 0.2971153259277344 seconds.
648518346349509855
Segment 648518346349509855 vertex labels regenerated in: 0.48233556747436523 seconds.
648518346349511212
Segment 648518346349511212 vertex labels re

In [5]:
#(schema.jobs & "table_name='__coarse_label_final'").delete()